In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetics for the plots
sns.set(style='whitegrid')

# Load the dataset
file_path = 'Community_Safety_Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()

In [ ]:
# Visualizations for categorical data

# Plotting the distribution of 'case_type_pubtrans'
plt.figure(figsize=(10, 8))
sns.countplot(y=data['case_type_pubtrans'], order = data['case_type_pubtrans'].value_counts().index[:10])
plt.title('Top 10 Case Types in Public Transportation')
plt.xlabel('Count')
plt.ylabel('Case Type')
plt.show()

# Plotting the distribution of 'municipality'
plt.figure(figsize=(10, 8))
sns.countplot(y=data['municipality'], order = data['municipality'].value_counts().index)
plt.title('Distribution of Cases by Municipality')
plt.xlabel('Count')
plt.ylabel('Municipality')
plt.show()

In [ ]:
# To identify the district with the highest risk of crime increase, we'll need to analyze the trend over time.
# For this, we'll group the data by 'municipality' and the time period (e.g., year, month) and then count the number of cases.

# The 'occ_date' column is not present. Instead, we have 'year', 'month', and 'day' columns already provided.
# Group by 'municipality', 'year' and 'month' and count the cases
crime_trends = data.groupby(['municipality', 'year', 'month']).size().reset_index(name='case_count')

# Now we'll calculate the percentage change in case count month-over-month to identify trends
crime_trends['percent_change'] = crime_trends.groupby(['municipality'])['case_count'].pct_change()

# To identify the district with the highest risk, we'll look for the municipality with the highest average monthly percentage increase
high_risk_district = crime_trends.groupby('municipality')['percent_change'].mean().reset_index()
high_risk_district = high_risk_district.sort_values(by='percent_change', ascending=False)

# Display the district with the highest average monthly percentage increase
high_risk_district.head(1)

In [ ]:
# Temporal Analysis for the municipality of King

# Filter the dataset for the municipality of King
data_king = crime_trends[crime_trends['municipality'] == 'King']

# Plotting the time series of crime rates
plt.figure(figsize=(15, 7))
sns.lineplot(x='month', y='case_count', data=data_king, estimator='sum', ci=None)
plt.title('Monthly Crime Rates in King')
plt.xlabel('Month')
plt.ylabel('Total Number of Cases')
plt.xticks(np.arange(1, 13, step=1))
plt.show()

In [ ]:
import itertools

# Preparing data for ARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# Filter the dataset for the municipality of King and drop the 'percent_change' as it's not needed for ARIMA
king_data = crime_trends[crime_trends['municipality'] == 'King'][['year', 'month', 'case_count']].copy()

# Creating a date index for the time series data
king_data['date'] = pd.to_datetime(king_data.assign(day=1).loc[:, ['year', 'month', 'day']])
king_data.set_index('date', inplace=True)
king_data.drop(['year', 'month'], axis=1, inplace=True)

# Check for missing months and fill if any
king_data = king_data.asfreq('MS').fillna(method='ffill')

# Decomposing the time series to observe trend, seasonality, and residuals
result = seasonal_decompose(king_data['case_count'], model='additive')
result.plot()
plt.show()

# Define the p, d, and q parameters to take any seasonal effects into account
p = d = q = range(0, 2)

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

# Define seasonal parameter
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

# Grid search to find the optimal set of parameters that yields the best performance for our model
best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
best_model = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(king_data['case_count'],
                          order=param,
                          seasonal_order=param_seasonal,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

            results = mod.fit()

            # Compare this model's AIC with our best model so far
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
                best_model = results
        except:
            continue

print('Best SARIMAX{}x{}12 model - AIC:{}'.format(best_pdq, best_seasonal_pdq, best_aic))

In [ ]:
# Fitting the SARIMAX model with the best parameters found

# Assuming the best parameters are stored in best_pdq and best_seasonal_pdq
# (Please replace these with the actual best parameters found from the output)
best_pdq = (1, 1, 1)
best_seasonal_pdq = (1, 1, 1, 12)

# Fit the SARIMAX model
best_model = SARIMAX(king_data['case_count'],
                    order=best_pdq,
                    seasonal_order=best_seasonal_pdq,
                    enforce_stationarity=False,
                    enforce_invertibility=False).fit()

# Summary of the model
print(best_model.summary())

# Get forecast for the next 12 months
forecast = best_model.get_forecast(steps=12)
forecast_ci = forecast.conf_int()

# Plot the forecast alongside historical data
plt.figure(figsize=(15, 7))
plt.plot(king_data['case_count'], label='Historical')
plt.plot(forecast.predicted_mean, label='Forecast')
plt.fill_between(forecast_ci.index,
                 forecast_ci.iloc[:, 0],
                 forecast_ci.iloc[:, 1], color='k', alpha=.25)
plt.title('Forecast of Monthly Crime Rates in King')
plt.xlabel('Date')
plt.ylabel('Total Number of Cases')
plt.legend()
plt.show()

In [ ]:
# To identify all districts with a risk of crime increase, we will calculate the average monthly percentage change in crime rates for each district.

# Group by 'municipality' and calculate the mean percentage change
risk_by_district = crime_trends.groupby('municipality')['percent_change'].mean().reset_index()

# Sort the districts by the average percentage change in descending order to see which have the highest increase
risk_by_district = risk_by_district.sort_values(by='percent_change', ascending=False)

# Display the sorted list of districts with their associated risk
risk_by_district

In [ ]:
# Load the staffing exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_df = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the dataframe
display(staffing_exceptions_df.head())

# Basic statistics and counts to identify patterns
staffing_summary = staffing_exceptions_df.describe(include='all')

# Display summary statistics
display(staffing_summary)

In [ ]:
# Import necessary libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Set the aesthetic style of the plots
sns.set_style('whitegrid')

# Plot the distribution of exceptions over time
plt.figure(figsize=(14, 7))
sns.countplot(data=staffing_exceptions_df, x='year')
plt.title('Distribution of Staffing Exceptions Over Years')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

# Plot the frequency of different types of exceptions
plt.figure(figsize=(14, 7))
sns.countplot(data=staffing_exceptions_df, y='Exception Type', order = staffing_exceptions_df['Exception Type'].value_counts().index)
plt.title('Frequency of Different Types of Staffing Exceptions')
plt.xlabel('Count')
plt.ylabel('Exception Type')
plt.show()

# Plot trends or anomalies in the number of hours for exceptions
plt.figure(figsize=(14, 7))
sns.boxplot(data=staffing_exceptions_df, x='year', y='# of Hours')
plt.title('Trends or Anomalies in the Number of Hours for Exceptions Over Years')
plt.xlabel('Year')
plt.ylabel('Number of Hours')
plt.show()

# Correlation between platoons and exception types
plt.figure(figsize=(14, 7))
sns.countplot(data=staffing_exceptions_df, y='Platoon', hue='Exception Type', order = staffing_exceptions_df['Platoon'].value_counts().index)
plt.title('Correlation Between Platoons and Exception Types')
plt.xlabel('Count')
plt.ylabel('Platoon')
plt.legend(title='Exception Type', loc='upper right')
plt.show()

In [ ]:
# Load the Calls for Service data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
calls_for_service_df = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the dataframe
display(calls_for_service_df.head())

# Verify the column names for date and time information
column_names = calls_for_service_df.columns.tolist()
display(column_names)

In [ ]:
# Import necessary libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Set the aesthetic style of the plots
sns.set_style('whitegrid')

# Plot the distribution of calls over years
plt.figure(figsize=(14, 7))
sns.countplot(data=calls_for_service_df, x='year')
plt.title('Distribution of Calls Over Years')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

# Plot the distribution of calls over months
plt.figure(figsize=(14, 7))
sns.countplot(data=calls_for_service_df, x='month')
plt.title('Distribution of Calls Over Months')
plt.xlabel('Month')
plt.ylabel('Count')
plt.show()

# Plot the distribution of calls over days of the week
plt.figure(figsize=(14, 7))
sns.countplot(data=calls_for_service_df, x='dayofweek')
plt.title('Distribution of Calls Over Days of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Count')
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.show()

# Plot the distribution of calls over quarters
plt.figure(figsize=(14, 7))
sns.countplot(data=calls_for_service_df, x='quarter')
plt.title('Distribution of Calls Over Quarters')
plt.xlabel('Quarter')
plt.ylabel('Count')
plt.show()

# Plot the frequency of different types of calls
plt.figure(figsize=(14, 7))
sns.countplot(data=calls_for_service_df, y='Call Type', order = calls_for_service_df['Call Type'].value_counts().index)
plt.title('Frequency of Different Types of Calls')
plt.xlabel('Count')
plt.ylabel('Call Type')
plt.show()

In [ ]:
# Aggregate the calls data to get the total number of calls for each day
calls_per_day = calls_for_service_df.groupby(calls_for_service_df['Call Date Time'].dt.date).size().reset_index(name='calls_count')

# Aggregate the staffing exceptions data to get the total number of hours of exceptions for each day
# Assuming there is a 'Date' column in the staffing exceptions data that represents the day of the exception
staffing_exceptions_per_day = staffing_exceptions_df.groupby(staffing_exceptions_df['Exception Date'].dt.date)['# of Hours'].sum().reset_index(name='exceptions_hours')

# Merge the two datasets on the day
demand_staffing_correlation = pd.merge(calls_per_day, staffing_exceptions_per_day, left_on='Call Date Time', right_on='Exception Date', how='inner')

# Calculate the correlation
correlation = demand_staffing_correlation[['calls_count', 'exceptions_hours']].corr()

# Display the correlation
print(correlation)

# Visualize the relationship with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=demand_staffing_correlation, x='calls_count', y='exceptions_hours')
plt.title('Correlation between Call Demand and Staffing Exceptions')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Aggregate the calls data to get the total number of calls for each day
calls_per_day = data.groupby(data['day']).size().reset_index(name='calls_count')

# Assuming there is a 'Date' column in the staffing exceptions data that represents the day of the exception
# Aggregate the staffing exceptions data to get the total number of hours of exceptions for each day
# For this example, we will create a dummy staffing exceptions dataframe
staffing_exceptions_data = {'Date': ['2021-01-01', '2021-01-02', '2021-01-03'], '# of Hours': [5, 3, 4]}
staffing_exceptions_df = pd.DataFrame(staffing_exceptions_data)
staffing_exceptions_df['Date'] = pd.to_datetime(staffing_exceptions_df['Date'])
staffing_exceptions_per_day = staffing_exceptions_df.groupby(staffing_exceptions_df['Date'].dt.date)['# of Hours'].sum().reset_index(name='exceptions_hours')

# Merge the two datasets on the day
demand_staffing_correlation = pd.merge(calls_per_day, staffing_exceptions_per_day, left_on='day', right_on='Date', how='inner')

# Calculate the correlation
correlation = demand_staffing_correlation[['calls_count', 'exceptions_hours']].corr()

# Display the correlation
print(correlation)

# Visualize the relationship with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=demand_staffing_correlation, x='calls_count', y='exceptions_hours')
plt.title('Correlation between Call Demand and Staffing Exceptions')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('/mnt/data/District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load a small sample of the Staffing Exceptions data to inspect the column names
df_sample = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)
df_sample.head()

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    chunk = chunk.groupby(chunk['Date']).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Merge the call data with the staffing exceptions data on the date
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='inner')

# Calculate the correlation between the total calls and the total # of hours of staffing exceptions
correlation = calls_and_staffing[['Total Calls', '# of Hours']].corr()

# Display the correlation matrix
correlation

In [ ]:
# Load the Calls for Service data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
call_chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    # Combine year, month, and day columns to create a 'Call Date' column
    chunk['Call Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    chunk = chunk.groupby(chunk['Call Date']).agg({'Total Calls': 'sum'}).reset_index()
    call_chunks.append(chunk)

# Concatenate chunks into a single dataframe
calls_aggregated = pd.concat(call_chunks, ignore_index=True)

# Display the first few rows of the aggregated call data
display(calls_aggregated.head())

In [ ]:
# Load a small sample of the Calls for Service data to inspect the column names
df_sample_calls = pd.read_csv('Calls_for_Service.csv', nrows=5)
df_sample_calls.head()

In [ ]:
# Load the Calls for Service data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
call_chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    # Combine year, month, and day columns to create a 'Call Date' column
    chunk['Call Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Count the number of calls for each date
    chunk = chunk.groupby(chunk['Call Date']).size().reset_index(name='Total Calls')
    call_chunks.append(chunk)

# Concatenate chunks into a single dataframe
calls_aggregated = pd.concat(call_chunks, ignore_index=True)

# Display the first few rows of the aggregated call data
display(calls_aggregated.head())

In [ ]:
# Merge the call data with the staffing exceptions data on the date
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='inner')

# Calculate the correlation between the total calls and the total # of hours of staffing exceptions
correlation = calls_and_staffing[['Total Calls', '# of Hours']].corr()

# Display the correlation matrix
correlation

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk['Date'] = chunk['Exception Date'].dt.date
    chunk_summary = chunk.groupby('Date').agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk_summary)

# Combine the results into a single dataframe
staffing_exceptions_summary = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the summary dataframe
staffing_exceptions_summary.head()

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size, usecols=['Date', '# of Hours']):
    # Process each chunk to extract necessary information
    chunk['Date'] = pd.to_datetime(chunk['Date'])
    chunk_summary = chunk.groupby(chunk['Date'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk_summary)

# Combine the results into a single dataframe
staffing_exceptions_summary = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the summary dataframe
staffing_exceptions_summary.head()

In [ ]:
# Load just the first few rows of the Staffing Exceptions data to inspect the column names
try:
    sample_chunk = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)
    print(sample_chunk.columns.tolist())
except Exception as e:
    print(f'Error: {e}')

In [ ]:
# Load the Staffing Exceptions data in chunks using the correct column names
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size, usecols=['Exception Start Time', '# of Hours']):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Start Time']).dt.date
    chunk_summary = chunk.groupby('Exception Date').agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk_summary)

# Combine the results into a single dataframe
staffing_exceptions_summary = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the summary dataframe
staffing_exceptions_summary.head()

In [ ]:
# Merge the call demand data with the staffing exceptions summary on the date
demand_staffing_merged = pd.merge(calls_per_day, staffing_exceptions_summary, left_on='day', right_on='Exception Date', how='inner')

# Calculate the correlation between call demand and staffing exceptions
# We will use Pearson correlation coefficient
correlation = demand_staffing_merged[['calls_count', '# of Hours']].corr(method='pearson')

# Display the correlation matrix
print(correlation)

# Visualize the relationship with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=demand_staffing_merged, x='calls_count', y='# of Hours')
plt.title('Correlation between Call Demand and Staffing Exceptions')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Check for missing or infinite values in the merged data
demand_staffing_merged.isnull().sum()

In [ ]:
# Inspect the unique values and range for 'calls_count' and '# of Hours'
print('Unique values in calls_count:', demand_staffing_merged['calls_count'].unique())
print('Unique values in # of Hours:', demand_staffing_merged['# of Hours'].unique())

# Check the range of values
print('Range of calls_count:', (demand_staffing_merged['calls_count'].min(), demand_staffing_merged['calls_count'].max()))
print('Range of # of Hours:', (demand_staffing_merged['# of Hours'].min(), demand_staffing_merged['# of Hours'].max()))

In [ ]:
# Inspect the calls data before merging
calls_per_day.describe(include='all')

# Inspect the staffing exceptions data before merging
staffing_exceptions_summary.describe(include='all')

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('/mnt/data/District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

# Now we will merge this with the calls data and perform the correlation analysis
# This will be done in the next cell to keep the operations modular and manageable.

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
staffing_exceptions_aggregated.head()

In [ ]:
# Inspect the first few rows of the CSV file to understand its structure
import pandas as pd

# Load the first few rows of the data
staffing_exceptions_sample = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)

# Display the first few rows to inspect the column names
staffing_exceptions_sample

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Group by the new 'Date' column and sum the '# of Hours'
    chunk = chunk.groupby('Date').agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
staffing_exceptions_aggregated.head()

In [ ]:
# Merge the aggregated staffing exceptions data with the calls data
# First, load the calls data

# Define a function to parse dates
parse_dates = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

calls_data = pd.read_csv('Calls_for_Service.csv', parse_dates=['Date'], date_parser=parse_dates)

# Group the calls data by date and count the number of calls
calls_data_grouped = calls_data.groupby('Date').size().reset_index(name='Number of Calls')

# Merge the two datasets on the 'Date' column
merged_data = pd.merge(staffing_exceptions_aggregated, calls_data_grouped, on='Date', how='outer')

# Fill NaN values with 0 for days with no staffing exceptions
merged_data.fillna({'# of Hours': 0, 'Number of Calls': 0}, inplace=True)

# Calculate the correlation between the number of hours of staffing exceptions and the number of calls
correlation = merged_data[['# of Hours', 'Number of Calls']].corr()

# Display the correlation matrix
merged_data.head(), correlation

In [ ]:
# Inspect the first few rows of the 'Calls_for_Service.csv' to understand its structure
import pandas as pd

# Load the first few rows of the data
calls_sample = pd.read_csv('Calls_for_Service.csv', nrows=5)

# Display the first few rows to inspect the column names
calls_sample

In [ ]:
# Load the Calls for Service data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Group by the new 'Date' column and count the number of calls
    chunk = chunk.groupby('Date').size().reset_index(name='Number of Calls')
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
calls_data_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated calls data
calls_data_aggregated.head()

In [ ]:
# Merge the aggregated calls data with the staffing exceptions data

# Merge the two datasets on the 'Date' column
merged_data = pd.merge(calls_data_aggregated, staffing_exceptions_aggregated, on='Date', how='outer')

# Fill NaN values with 0 for days with no staffing exceptions or calls
merged_data.fillna({'# of Hours': 0, 'Number of Calls': 0}, inplace=True)

# Calculate the correlation between the number of hours of staffing exceptions and the number of calls
correlation = merged_data[['# of Hours', 'Number of Calls']].corr()

# Display the correlation matrix
merged_data.head(), correlation

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('/mnt/data/District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Convert 'Date' to datetime and extract the date part
    chunk['Date'] = pd.to_datetime(chunk['Date']).dt.date
    # Aggregate the data by 'Date' to get the total number of hours of exceptions for each day
    daily_exceptions = chunk.groupby('Date')['# of Hours'].sum().reset_index()
    chunks.append(daily_exceptions)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)
# Aggregate again in case the same date appears in different chunks
staffing_exceptions_aggregated = staffing_exceptions_aggregated.groupby('Date')['# of Hours'].sum().reset_index()

# Display the first few rows of the aggregated exceptions dataframe
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Convert 'Date' to datetime and extract the date part
    chunk['Date'] = pd.to_datetime(chunk['Date']).dt.date
    # Aggregate the data by 'Date' to get the total number of hours of exceptions for each day
    daily_exceptions = chunk.groupby('Date')['# of Hours'].sum().reset_index()
    chunks.append(daily_exceptions)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)
# Aggregate again in case the same date appears in different chunks
staffing_exceptions_aggregated = staffing_exceptions_aggregated.groupby('Date')['# of Hours'].sum().reset_index()

# Display the first few rows of the aggregated exceptions dataframe
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load the first chunk of the Staffing Exceptions data to inspect the column names
chunk_size = 50000  # Define the chunk size
first_chunk = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size).__next__()

# Display the column names
first_chunk.columns.tolist()

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('/mnt/data/District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract the necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).size().reset_index(name='exceptions_count')
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract the necessary information
    chunk['Exception Date'] = pd.to_datetime(chunk['Exception Date'])
    chunk = chunk.groupby(chunk['Exception Date'].dt.date).size().reset_index(name='exceptions_count')
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
staffing_exceptions_aggregated.head()

In [ ]:
# Let's read the first few lines of the 'District_Platoon_Staffing_Exceptions.csv' to understand its structure
import pandas as pd

# Read in the first few rows of the dataframe
df_head = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)

# Display the first few rows
df_head

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract the necessary information
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Group by the new 'Date' column and sum the '# of Hours'
    chunk = chunk.groupby('Date')['# of Hours'].sum().reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
staffing_exceptions_aggregated.head()

In [ ]:
# Merge the calls data with the staffing exceptions data on the date
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Date', right_on='Date', how='left')

# Fill NaN values with 0 for days where there are no staffing exceptions
# This assumes that a NaN value means no exceptions were recorded for that day
calls_and_staffing['# of Hours'].fillna(0, inplace=True)

# Display the first few rows of the merged dataframe
calls_and_staffing.head()

In [ ]:
# Ensure the 'Date' column in calls_aggregated is of datetime type for merging
calls_aggregated['Date'] = pd.to_datetime(calls_aggregated['Date'])

# Merge the calls data with the staffing exceptions data on the date
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Date', right_on='Date', how='left')

# Fill NaN values with 0 for days where there are no staffing exceptions
# This assumes that a NaN value means no exceptions were recorded for that day
calls_and_staffing['# of Hours'].fillna(0, inplace=True)

# Display the first few rows of the merged dataframe
calls_and_staffing.head()

In [ ]:
# Check if 'Date' column exists in calls_aggregated and staffing_exceptions_aggregated
try:
    calls_aggregated['Date']
except KeyError:
    # If 'Date' column doesn't exist, create it from 'year', 'month', and 'day'
    calls_aggregated['Date'] = pd.to_datetime(calls_aggregated[['year', 'month', 'day']])

try:
    staffing_exceptions_aggregated['Date']
except KeyError:
    # If 'Date' column doesn't exist, create it from 'year', 'month', and 'day'
    staffing_exceptions_aggregated['Date'] = pd.to_datetime(staffing_exceptions_aggregated[['year', 'month', 'day']])

# Now that we have ensured both dataframes have a 'Date' column, attempt the merge again
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Date', right_on='Date', how='left')

# Fill NaN values with 0 for days where there are no staffing exceptions
# This assumes that a NaN value means no exceptions were recorded for that day
calls_and_staffing['exceptions_count'].fillna(0, inplace=True)

# Display the first few rows of the merged dataframe
calls_and_staffing.head()

In [ ]:
# Display column names for both dataframes to understand their structure
print('Calls Aggregated Columns:\n', calls_aggregated.columns)
print('\nStaffing Exceptions Aggregated Columns:\n', staffing_exceptions_aggregated.columns)

In [ ]:
# Merge the calls data with the staffing exceptions data on the correct date columns
calls_and_staffing = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='left')

# Fill NaN values with 0 for days where there are no staffing exceptions
# This assumes that a NaN value means no exceptions were recorded for that day
calls_and_staffing['# of Hours'].fillna(0, inplace=True)

# Display the first few rows of the merged dataframe
calls_and_staffing.head()

In [ ]:
# Calculate the correlation coefficient between 'Total Calls' and '# of Hours' to understand the relationship
correlation = calls_and_staffing['Total Calls'].corr(calls_and_staffing['# of Hours'])
print('Correlation coefficient:', correlation)

In [ ]:
# Define a function to process chunks of the staffing exceptions data
def process_staffing_exceptions_chunk(chunk):
    # Convert 'Date' to datetime and extract the date part
    chunk['Date'] = pd.to_datetime(chunk['Date']).dt.date
    # Group by 'Date' and sum the '# of Hours'
    return chunk.groupby('Date')['# of Hours'].sum().reset_index()

# Initialize an empty dataframe to store aggregated results
staffing_exceptions_aggregated = pd.DataFrame()

# Define the chunk size
chunk_size = 50000

# Read the staffing exceptions file in chunks and process each chunk
for chunk in pd.read_csv('/mnt/data/District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    staffing_exceptions_aggregated = staffing_exceptions_aggregated.append(process_staffing_exceptions_chunk(chunk))

# Group by 'Date' again in case there are overlapping dates between chunks
staffing_exceptions_aggregated = staffing_exceptions_aggregated.groupby('Date').sum().reset_index()

# Display the first few rows of the aggregated staffing exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Define a function to process chunks of the staffing exceptions data
def process_staffing_exceptions_chunk(chunk):
    # Convert 'Date' to datetime and extract the date part
    chunk['Date'] = pd.to_datetime(chunk['Date']).dt.date
    # Group by 'Date' and sum the '# of Hours'
    return chunk.groupby('Date')['# of Hours'].sum().reset_index()

# Initialize an empty dataframe to store aggregated results
staffing_exceptions_aggregated = pd.DataFrame()

# Define the chunk size
chunk_size = 50000

# Read the staffing exceptions file in chunks and process each chunk
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    staffing_exceptions_aggregated = staffing_exceptions_aggregated.append(process_staffing_exceptions_chunk(chunk))

# Group by 'Date' again in case there are overlapping dates between chunks
staffing_exceptions_aggregated = staffing_exceptions_aggregated.groupby('Date').sum().reset_index()

# Display the first few rows of the aggregated staffing exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load a small sample of the staffing exceptions data to check the structure and column names
staffing_exceptions_sample = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)
staffing_exceptions_sample.head()

In [ ]:
# Define a function to process chunks of the staffing exceptions data
def process_staffing_exceptions_chunk(chunk):
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Group by 'Date' and sum the '# of Hours'
    return chunk.groupby('Date')['# of Hours'].sum().reset_index()

# Initialize an empty dataframe to store aggregated results
staffing_exceptions_aggregated = pd.DataFrame()

# Define the chunk size
chunk_size = 50000

# Read the staffing exceptions file in chunks and process each chunk
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    staffing_exceptions_aggregated = staffing_exceptions_aggregated.append(process_staffing_exceptions_chunk(chunk))

# Group by 'Date' again in case there are overlapping dates between chunks
staffing_exceptions_aggregated = staffing_exceptions_aggregated.groupby('Date').sum().reset_index()

# Display the first few rows of the aggregated staffing exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Merge the calls for service data with the staffing exceptions data on the 'Date' column
calls_staffing_merged = calls_for_service_aggregated.merge(staffing_exceptions_aggregated, on='Date', how='inner')

# Calculate the correlation between 'Total Calls' and '# of Hours'
correlation = calls_staffing_merged[['Total Calls', '# of Hours']].corr()

# Display the correlation matrix
correlation

In [ ]:
# Load a small sample of the calls for service data to check the structure and column names
calls_for_service_sample = pd.read_csv('Calls_for_Service.csv', nrows=5)
calls_for_service_sample.head()

In [ ]:
# Define a function to process chunks of the calls for service data
def process_calls_for_service_chunk(chunk):
    # Combine year, month, and day columns to create a 'Date' column
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Count the number of calls for each date
    return chunk.groupby('Date').size().reset_index(name='Total Calls')

# Initialize an empty dataframe to store aggregated results
calls_for_service_aggregated = pd.DataFrame()

# Define the chunk size
chunk_size = 50000

# Read the calls for service file in chunks and process each chunk
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    calls_for_service_aggregated = calls_for_service_aggregated.append(process_calls_for_service_chunk(chunk))

# Group by 'Date' again in case there are overlapping dates between chunks
calls_for_service_aggregated = calls_for_service_aggregated.groupby('Date').sum().reset_index()

# Display the first few rows of the aggregated calls for service data
display(calls_for_service_aggregated.head())

In [ ]:
# Merge the calls for service data with the staffing exceptions data on the 'Date' column
calls_staffing_merged = calls_for_service_aggregated.merge(staffing_exceptions_aggregated, on='Date', how='inner')

# Calculate the correlation between 'Total Calls' and '# of Hours'
correlation = calls_staffing_merged[['Total Calls', '# of Hours']].corr()

# Display the correlation matrix
correlation

In [ ]:
# Load the District_Platoon_Staffing_Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Process each chunk to extract necessary information
    chunk['Exception Start Time'] = pd.to_datetime(chunk['Exception Start Time'])
    chunk = chunk.groupby(chunk['Exception Start Time'].dt.date).agg({'# of Hours': 'sum'}).reset_index()
    chunks.append(chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load the first chunk of the District_Platoon_Staffing_Exceptions data to inspect the column names
chunk_size = 50000  # Define the chunk size

# Read the first chunk of the file
first_chunk = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size).__next__()

# Display the column names of the first chunk
column_names = first_chunk.columns.tolist()
display(column_names)

In [ ]:
# Merge the calls for service data with the staffing exceptions data on the date
calls_for_service_df['Call Date'] = pd.to_datetime(calls_for_service_df['Call Date'])
merged_data = pd.merge(calls_for_service_df, staffing_exceptions_aggregated, left_on='Call Date', right_on='Exception Start Time', how='inner')

# Calculate the total number of calls per day
calls_per_day = merged_data.groupby('Call Date').size().reset_index(name='calls_count')

# Merge the calls per day with the exceptions hours per day
demand_staffing_correlation = pd.merge(calls_per_day, staffing_exceptions_aggregated, left_on='Call Date', right_on='Exception Start Time', how='inner')

# Calculate the correlation
correlation = demand_staffing_correlation[['calls_count', '# of Hours']].corr()

# Display the correlation matrix
display(correlation)

# Visualize the correlation with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=demand_staffing_correlation, x='calls_count', y='# of Hours')
plt.title('Correlation between Call Demand and Staffing Exceptions Hours')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Load the Community_Safety_Data.csv into a dataframe
data = pd.read_csv('Community_Safety_Data.csv')

# Convert the 'year', 'month', and 'day' columns to a datetime object
data['Call Date'] = pd.to_datetime(data[['year', 'month', 'day']])

# Display the first few rows to confirm the 'Call Date' column
data[['Call Date', 'case_type_pubtrans', 'municipality']].head()

In [ ]:
# Ensure both 'Call Date' and 'Exception Start Time' are of the same data type before merging
data['Call Date'] = pd.to_datetime(data['Call Date'])
staffing_exceptions_aggregated['Exception Start Time'] = pd.to_datetime(staffing_exceptions_aggregated['Exception Start Time'])

# Merge the calls for service data with the staffing exceptions data on the date
merged_data = pd.merge(data, staffing_exceptions_aggregated, left_on='Call Date', right_on='Exception Start Time', how='inner')

# Calculate the total number of calls per day
calls_per_day = merged_data.groupby('Call Date').size().reset_index(name='calls_count')

# Merge the calls per day with the exceptions hours per day
demand_staffing_correlation = pd.merge(calls_per_day, staffing_exceptions_aggregated, left_on='Call Date', right_on='Exception Start Time', how='inner')

# Calculate the correlation
correlation = demand_staffing_correlation[['calls_count', '# of Hours']].corr()

# Display the correlation matrix
display(correlation)

# Visualize the correlation with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=demand_staffing_correlation, x='calls_count', y='# of Hours')
plt.title('Correlation between Call Demand and Staffing Exceptions Hours')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Check for overlapping dates between the two datasets
calls_dates = data['Call Date'].dt.date.unique()
exceptions_dates = staffing_exceptions_aggregated['Exception Start Time'].dt.date.unique()
overlapping_dates = set(calls_dates).intersection(set(exceptions_dates))

# Display the number of overlapping dates
len(overlapping_dates)

In [ ]:
# Load the Staffing Exceptions data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('District_Platoon_Staffing_Exceptions.csv', chunksize=chunk_size):
    # Create a 'Date' column by combining 'year', 'month', and 'day'
    chunk['Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Aggregate the data by Date
    aggregated_chunk = chunk.groupby('Date')['# of Hours'].sum().reset_index(name='exceptions_hours')
    chunks.append(aggregated_chunk)

# Concatenate chunks into a single dataframe
staffing_exceptions_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated exceptions data
display(staffing_exceptions_aggregated.head())

In [ ]:
# Load a small sample of the Staffing Exceptions data to check the column names
sample = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)

# Display the column names
sample.columns.tolist()

In [ ]:
# Merge the call demand data with the staffing exceptions data
merged_data = pd.merge(calls_per_day, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='inner')

# Calculate the correlation between calls count and exceptions hours
correlation = merged_data[['calls_count', 'exceptions_hours']].corr()

# Display the correlation matrix
display(correlation)

# Visualize the correlation with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_data, x='calls_count', y='exceptions_hours')
plt.title('Correlation between Call Demand and Staffing Exceptions Hours')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Load the Calls for Service data in chunks due to its large size
chunk_size = 50000  # Define the chunk size
chunks = []  # List to hold chunks of dataframes

# Read the file in chunks
for chunk in pd.read_csv('Calls_for_Service.csv', chunksize=chunk_size):
    # Create a 'Call Date' column by combining 'year', 'month', and 'day'
    chunk['Call Date'] = pd.to_datetime(chunk[['year', 'month', 'day']])
    # Aggregate the data by Call Date
    aggregated_chunk = chunk.groupby('Call Date').size().reset_index(name='calls_count')
    chunks.append(aggregated_chunk)

# Concatenate chunks into a single dataframe
calls_aggregated = pd.concat(chunks, ignore_index=True)

# Display the first few rows of the aggregated calls data
display(calls_aggregated.head())

In [ ]:
# Load a small sample of the Calls for Service data to check the column names
sample = pd.read_csv('Calls_for_Service.csv', nrows=5)

# Display the column names
sample.columns.tolist()

In [ ]:
# Merge the call demand data with the staffing exceptions data
merged_data = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='inner')

# Calculate the correlation between calls count and exceptions hours
correlation = merged_data[['calls_count', 'exceptions_hours']].corr()

# Display the correlation matrix
display(correlation)

# Visualize the correlation with a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_data, x='calls_count', y='exceptions_hours')
plt.title('Correlation between Call Demand and Staffing Exceptions Hours')
plt.xlabel('Total Calls per Day')
plt.ylabel('Total Staffing Exceptions Hours per Day')
plt.show()

In [ ]:
# Prepare the data for time-series forecasting
# We will use the 'Date' from the staffing exceptions data as the index
staffing_data_for_model = staffing_exceptions_aggregated.set_index('Date')
staffing_data_for_model.sort_index(inplace=True)

# Check for missing dates and fill them with the previous value (forward-fill)
staffing_data_for_model = staffing_data_for_model.asfreq('D', method='ffill')

# Visualize the staffing exceptions hours over time
plt.figure(figsize=(15, 7))
plt.plot(staffing_data_for_model.index, staffing_data_for_model['exceptions_hours'], label='Staffing Exceptions Hours')
plt.title('Staffing Exceptions Hours Over Time')
plt.xlabel('Date')
plt.ylabel('Total Staffing Exceptions Hours')
plt.legend()
plt.show()

# We will use the ARIMA model for time-series forecasting
# First, we need to find the optimal parameters for the ARIMA model
import pmdarima as pm

# Use auto_arima to find the best parameters
arima_model = pm.auto_arima(staffing_data_for_model['exceptions_hours'], seasonal=True, m=7, suppress_warnings=True)

# Display the summary of the ARIMA model
arima_model.summary()

In [ ]:
!pip install -q pmdarima

In [ ]:
# Now that pmdarima is installed, let's try fitting the ARIMA model again

# Use auto_arima to find the best parameters
arima_model = pm.auto_arima(staffing_data_for_model['exceptions_hours'], seasonal=True, m=7, suppress_warnings=True)

# Display the summary of the ARIMA model
arima_model.summary()

In [ ]:
# Import the pmdarima library
import pmdarima as pm

# Now that pmdarima is installed, let's try fitting the ARIMA model again

# Use auto_arima to find the best parameters
arima_model = pm.auto_arima(staffing_data_for_model['exceptions_hours'], seasonal=True, m=7, suppress_warnings=True)

# Display the summary of the ARIMA model
arima_model.summary()

In [ ]:
# Merge the call demand data with the staffing exceptions data on the date
merged_data = pd.merge(calls_aggregated, staffing_exceptions_aggregated, left_on='Call Date', right_on='Date', how='inner')

# Calculate the correlation between calls count and exceptions hours
correlation = merged_data[['calls_count', 'exceptions_hours']].corr()

# Display the correlation matrix
display(correlation)

In [ ]:
# For predicting future crime trends and call demands, we will use the calls_aggregated data
# We will create a time series model using the calls data
calls_data_for_model = calls_aggregated.set_index('Call Date')
calls_data_for_model.sort_index(inplace=True)

# Check for missing dates and fill them with the previous value (forward-fill)
calls_data_for_model = calls_data_for_model.asfreq('D', method='ffill')

# Visualize the calls data over time
plt.figure(figsize=(15, 7))
plt.plot(calls_data_for_model.index, calls_data_for_model['calls_count'], label='Calls Count')
plt.title('Calls Count Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Calls')
plt.legend()
plt.show()

# We will use the ARIMA model for time-series forecasting
# First, we need to find the optimal parameters for the ARIMA model
import pmdarima as pm

# Use auto_arima to find the best parameters
arima_model_calls = pm.auto_arima(calls_data_for_model['calls_count'], seasonal=True, m=7, suppress_warnings=True)

# Display the summary of the ARIMA model for calls data
arima_model_calls.summary()

In [ ]:
# Handle duplicate dates in the calls_aggregated data
calls_aggregated_unique = calls_aggregated.groupby('Call Date').sum().reset_index()

# Set the index to 'Call Date' and sort by index
calls_data_for_model = calls_aggregated_unique.set_index('Call Date')
calls_data_for_model.sort_index(inplace=True)

# Now that we have a unique index, we can reapply the forward-fill method
calls_data_for_model = calls_data_for_model.asfreq('D', method='ffill')

# Retry visualizing the calls data over time
plt.figure(figsize=(15, 7))
plt.plot(calls_data_for_model.index, calls_data_for_model['calls_count'], label='Calls Count')
plt.title('Calls Count Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Calls')
plt.legend()
plt.show()

# Retry fitting the ARIMA model
# Use auto_arima to find the best parameters
arima_model_calls = pm.auto_arima(calls_data_for_model['calls_count'], seasonal=True, m=7, suppress_warnings=True)

# Display the summary of the ARIMA model for calls data
arima_model_calls.summary()

In [ ]:
# Import necessary libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Prepare the data for the machine learning model
# For this example, we will use the number of calls as a feature to predict staffing exceptions
X = merged_data[['calls_count']]  # Features
y = merged_data['exceptions_hours']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate the root mean squared error
rmse = sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)

In [ ]:
# Time-series forecasting for staffing challenges
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define the model
sarimax_model = SARIMAX(staffing_data_for_model['exceptions_hours'],
                        order=(1, 1, 1),
                        seasonal_order=(1, 1, 1, 7),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

# Fit the model
sarimax_result = sarimax_model.fit()

# Summary of the model
sarimax_summary = sarimax_result.summary()
sarimax_summary

In [ ]:
# Diagnostic checking of the SARIMAX model
sarimax_result.plot_diagnostics(figsize=(15, 12))
plt.show()

# Forecasting the next 30 days
forecast = sarimax_result.get_forecast(steps=30)
predicted_mean = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

# Plot the forecast along with the confidence interval
plt.figure(figsize=(15, 7))
plt.plot(staffing_data_for_model.index, staffing_data_for_model['exceptions_hours'], label='Observed')
plt.plot(predicted_mean.index, predicted_mean, label='Forecast', color='r')
plt.fill_between(predicted_mean.index,
                 confidence_intervals.iloc[:, 0],
                 confidence_intervals.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Staffing Exceptions Hours Forecast')
plt.xlabel('Date')
plt.ylabel('Staffing Exceptions Hours')
plt.legend()
plt.show()

In [ ]:
# Scheduling Optimization Recommendations
# Based on the forecasted staffing challenges, we can provide some recommendations.

# Identify days with the highest forecasted staffing exceptions
high_risk_days = predicted_mean[predicted_mean > predicted_mean.quantile(0.75)]

# Recommendations for scheduling
recommendations = []
for date, value in high_risk_days.iteritems():
    recommendations.append(f'On {date.date()}, consider increasing staff or having backup plans due to forecasted high staffing exceptions.')

# Display recommendations
for recommendation in recommendations:
    print(recommendation)